In [1]:
import keras
import datetime
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
# Setting seed for reproducability
np.random.seed(1234)  
PYTHONHASHSEED = 0
from sklearn import preprocessing
from sklearn.metrics import confusion_matrix, recall_score, precision_score
from keras import models
from keras.wrappers.scikit_learn import KerasRegressor
from keras.models import Sequential,model_from_json
from keras.layers import Dense, Dropout, LSTM, Activation
%matplotlib inline
from decimal import Decimal
import warnings
warnings.filterwarnings("ignore")
import pickle
from sklearn.preprocessing import OneHotEncoder,LabelEncoder,Normalizer, MinMaxScaler
from sklearn.cross_validation import cross_val_score
from numpy.random import seed
seed(1)
from tensorflow import set_random_seed
set_random_seed(2)
from sklearn.metrics import mean_absolute_error
from sklearn import ensemble
from sklearn.ensemble import RandomForestRegressor
from sklearn.multioutput import MultiOutputRegressor
import xgboost as xgb

Using TensorFlow backend.


In [2]:
df=pd.read_csv('C:\\Users\\lengada1\\NCSU\\DC_ten_skus.csv')
#df=pd.read_csv('./DC_ten_skus.csv')
df['Date'] = pd.to_datetime(df['Date'])
df['Year'] = df['Date'].dt.year
df['Month'] = df['Date'].dt.month
df['Day'] = df['Date'].dt.weekday_name
day_dummy=pd.get_dummies(df.Day)
df=pd.concat([df,day_dummy],axis=1)
df.drop(['Day','Date'],inplace=True,axis=1)

y=df['Sales']

In [3]:
df.head()
df.shape

(942, 16)

In [4]:
X=df.drop(['id','DayOfWeek','Customers','Sales'],inplace=False,axis=1)
X.head()

,Open,Promo,SchoolHoliday,Year,Month,Friday,Monday,Saturday,Sunday,Thursday,Tuesday,Wednesday
0,0,0,10,2013,1,0,0,0,0,0,1,0
1,10,0,10,2013,1,0,0,0,0,0,0,1
2,10,0,8,2013,1,0,0,0,0,1,0,0
3,10,0,8,2013,1,1,0,0,0,0,0,0
4,10,0,1,2013,1,0,0,1,0,0,0,0


In [5]:
for obs in range(1,8):
    X["Sales_T"+str(obs)]=df['Sales'].shift(obs)


In [6]:
X['Mov_avg']=df['Sales'].rolling( window=7).mean().shift(1) 
y=y[7:]
X=X[7:]
y.reset_index(drop=True, inplace=True)
X.reset_index(drop=True, inplace=True)
X.head()

,Open,Promo,SchoolHoliday,Year,Month,Friday,Monday,Saturday,Sunday,Thursday,Tuesday,Wednesday,Sales_T1,Sales_T2,Sales_T3,Sales_T4,Sales_T5,Sales_T6,Sales_T7,Mov_avg
0,10,10,1,2013,1,0,0,0,0,0,1,0,89391.0,0.0,43727.0,55606.0,52848.0,60436.0,0.0,43144.000000
1,10,10,1,2013,1,0,0,0,0,0,0,1,72552.0,89391.0,0.0,43727.0,55606.0,52848.0,60436.0,53508.571429
2,10,10,1,2013,1,0,0,0,0,1,0,0,64297.0,72552.0,89391.0,0.0,43727.0,55606.0,52848.0,54060.142857
3,10,10,1,2013,1,1,0,0,0,0,0,0,64112.0,64297.0,72552.0,89391.0,0.0,43727.0,55606.0,55669.285714
4,10,0,0,2013,1,0,0,1,0,0,0,0,65006.0,64112.0,64297.0,72552.0,89391.0,0.0,43727.0,57012.142857


In [7]:
std = preprocessing.StandardScaler().fit(X)
X = std.transform(X)
len(X)

935

In [8]:
cv=2

RF=RandomForestRegressor( n_estimators=1000,max_depth=10,max_features=14);
rf_models=[];
rf_models=(cross_val_score(RF,X,y,cv=cv, scoring ='mean_absolute_error').mean().round(0) )


GB=ensemble.GradientBoostingRegressor(n_estimators= 1000,max_depth= 8, max_features=14,
          learning_rate= 0.01, loss= 'ls');
gb_models=[];
gb_models=(cross_val_score(GB,X,y,cv=cv, scoring ='mean_absolute_error').mean().round(0)  )
   

XG=xgb.XGBRegressor(n_estimators= 900,learning_rate =0.01, max_depth=6,gamma=2)
xg_models=[]
xg_models=(cross_val_score(XG,X,y,cv=cv, scoring ='mean_absolute_error').mean().round(0)  )

In [9]:
def create_MLP():
    mlp=models.Sequential()
    mlp.add(Dense(20, input_dim=X.shape[1], activation='relu'))
    #mlp.add(Dense(20, activation='relu'))
    mlp.add(Dense(30, activation='relu'))
    mlp.add(Dense(10, activation='relu'))    
    mlp.add(Dense(1, activation='relu'))
    mlp.compile(loss='mae', optimizer='adam', metrics=['mae'])
    return mlp

MLP = KerasRegressor(build_fn=create_MLP,
                               epochs=300,
                               batch_size=20,
                               verbose=0)

mlp_models=[]
mlp_models=(cross_val_score(MLP,X,y,cv=cv).mean().round(0)  )

In [11]:
from sklearn.gaussian_process.kernels import RBF
from sklearn.gaussian_process.kernels import RationalQuadratic
from sklearn.gaussian_process.kernels import ExpSineSquared
from sklearn.gaussian_process.kernels import Matern
from sklearn.gaussian_process import GaussianProcessRegressor

kernels = [    
    1.0* RBF() + 
    1.0* RBF() *
    1.0* ExpSineSquared(periodicity=7.0,length_scale_bounds=(1,10),periodicity_bounds=(7, 7) )   
]

n_restarts_optimizer=10

GPR=GaussianProcessRegressor(kernel=kernels[0], n_restarts_optimizer=n_restarts_optimizer)  

gpr_models=[]
gpr_models=(cross_val_score(GPR,X,y,cv=cv, scoring='mean_absolute_error').mean().round(0))

In [12]:
#Summary
mae=-pd.DataFrame(np.column_stack((mlp_models,rf_models,gb_models,xg_models,gpr_models)) )
mae=mae.rename(columns = {0:'MLP',1:'RF',2:'GB',3:'XG',4:"GPR"})
best=mae.idxmin(axis=1)
best.astype('category')
mae["Best"]=best
mae

,MLP,RF,GB,XG,GPR,Best
0,29334.0,3306.0,3220.0,3208.0,51882.0,XG


In [13]:
#Same models as above, but configured here to predict over entire X (935 days)
def RF_final(X, y):
    tree=RF
    tree.fit(X,y)
    pred=tree.predict(X)
    pred=pd.DataFrame(pred)
    pred.reset_index(drop=True, inplace=True)
    pred=pred.rename(columns = {0:'RF'})
    return (pred, tree)

def MLP_final(X,y):
    model = MLP
    model.fit(X,y)
    pred=model.predict(X)
    pred=pd.DataFrame(pred)
    pred.reset_index(drop=True, inplace=True)
    pred=pred.rename(columns = {0:'MLP'})
    return (pred, model)

def GB_final(X,y):
    model = GB
    model.fit(X,y)
    prediction=model.predict(X)
    pred=pd.DataFrame(prediction)
    pred.reset_index(drop=True, inplace=True)
    pred=pred.rename(columns = {0:'GB'})
    return (pred, model)

def XG_final(X,y):
    model=XG
    model.fit(X,y)
    prediction=model.predict(X)
    pred=pd.DataFrame(prediction)
    pred.reset_index(drop=True, inplace=True)
    pred=pred.rename(columns = {0:'XG'})
    return (pred, model)

In [14]:
#Using aggregated SKU sales data, the best performing model is selected based on K-fold. 
#Then best model is used to predict full data set 

bm_dict={}
for sku in range(0,1):        
        if  mae["Best"][0]=="RF":
            bm_dict[sku]=RF_final(X,y);
                    
        elif mae["Best"][0]=="NN":
            bm_dict[sku]=MLP_final(X,y);
                                
        elif mae["Best"][0]=="GB":
            bm_dict[sku]=GB_final(X,y); 
                        
        elif mae["Best"][0]=="XG":
            bm_dict[sku]=XG_final(X,y);

In [15]:
# Create Phase 2 data set which is comprised of both aggregate SKU data and forecasts
dc_pred=pd.DataFrame(bm_dict[0][0])
# sku_pred=pd.read_csv('C:\\Users\\lengada1\\NCSU\\prediction_skus.csv')
sku_pred=pd.read_csv('./prediction_skus.csv')
sku_pred=sku_pred.iloc[:,1:]
X2=pd.concat([sku_pred,dc_pred,pd.DataFrame(X)],axis=1)
sku_pred

,1,2,3,4,5,6,7,8,9,10,Total
0,5717.818209,6301.653071,9865.725643,10153.291251,5602.367520,7176.216977,10341.561739,6505.311482,6286.657000,5714.657222,73665.260340
1,5510.298604,6512.066582,8556.524297,9330.420316,5765.830248,6921.244257,8090.852354,5673.728950,5216.300300,5539.029505,67116.295407
2,5079.047790,5460.737801,7735.243498,9324.782816,4944.436392,6314.656802,8066.825115,6234.547054,5395.656700,5616.178584,64172.112592
3,4855.066339,4907.510546,8600.125698,9873.201129,5069.712080,6927.614117,8099.700803,5206.362083,5717.865700,5606.811352,64863.969870
4,4848.817431,2609.258951,4299.631053,9935.107310,1835.048437,3765.943200,4538.281949,2510.754229,4943.071000,4541.323872,43827.237233
5,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.162793,0.000000,0.162793
6,4364.450817,4004.262165,5161.439033,7940.504973,3824.709259,5972.879755,6844.887051,4286.980709,4499.971700,4817.428118,51717.513560
7,4031.301953,3947.826898,5158.895142,7462.205307,3727.959016,5616.944218,6807.330886,4192.341320,4182.101000,4569.376346,49696.282161
8,3944.842050,4649.660839,5124.796593,7195.689326,3997.489338,5247.771443,6372.850673,3895.344289,3797.039300,4551.669918,48777.153776
9,3894.189293,3881.791308,5029.780984,7370.564596,3713.124132,5554.787497,6780.353349,4827.428066,4405.817400,4520.857199,49978.693807


In [16]:
rng=range(0,33)
new_cols =  ['' + str(i) for i in rng]
X2.columns = new_cols[:32]
std2= preprocessing.StandardScaler().fit(X2)
X2 = std2.transform(X2)

In [17]:
def create_MLP2():
    mlp=models.Sequential()
    mlp.add(Dense(40, input_dim=X2.shape[1], activation='relu'))
    #mlp.add(Dense(20, activation='relu'))
    mlp.add(Dense(30, activation='relu'))
    mlp.add(Dense(10, activation='relu'))    
    mlp.add(Dense(1, activation='relu'))
    mlp.compile(loss='mae', optimizer='adam', metrics=['mae'])
    return mlp

MLP2 = KerasRegressor(build_fn=create_MLP2,
                               epochs=600,
                               batch_size=20,
                               verbose=0)

In [18]:
rf2_models=[];
rf2_models=(cross_val_score(RF,X2,y,cv=cv, scoring ='mean_absolute_error').mean().round(0) )

gb2_models=[];
gb2_models=(cross_val_score(GB,X2,y,cv=cv, scoring ='mean_absolute_error').mean().round(0)  )
   

xg2_models=[];
xg2_models=(cross_val_score(XG,X2,y,cv=cv, scoring ='mean_absolute_error').mean().round(0)  )

In [19]:

mlp2_models=[];
mlp2_models=(cross_val_score(MLP2,X2,y,cv=cv, scoring='mean_absolute_error').mean().round(0)  )

In [20]:
mae2=-pd.DataFrame(np.column_stack((mlp2_models,rf2_models,gb2_models,xg2_models)) )
mae2=mae2.rename(columns = {0:'MLP',1:'RF',2:'GB',3:'XG'})
mae2

,MLP,RF,GB,XG
0,1340.0,1209.0,1179.0,1072.0


In [21]:
#Using forecasted data and no empirical sales data, the best performaning model is selected based on Train/Test. 
## NOTE this improvement is not the true delta.  Only a final forecast between Stage 1 vs 2 will tell. 

best2=mae2.idxmin(axis=1)
best2.astype('category')
mae2["Best"]=best2
both_mae=pd.concat([mae,mae2],axis=0)
both_mae.reset_index(drop=True, inplace=True)

# Phase 1 & 2 DC Models compared based on Test set MAE scores. 
both_mae['min']=both_mae.min(axis=1)
both_mae['error reduction']= ((both_mae['min'][0]-both_mae['min'][1])/both_mae['min'][0] ).round(3)
both_mae

,Best,GB,GPR,MLP,RF,XG,min,error reduction
0,XG,3220.0,51882.0,29334.0,3306.0,3208.0,3208.0,0.666
1,XG,1179.0,NaN,1340.0,1209.0,1072.0,1072.0,0.666


In [22]:
bm2_dict={}
for sku in range(0,1):
        if  mae2["Best"][0]=="RF":
            bm2_dict[sku]=RF_final(X2,y);
                    
        elif mae2["Best"][0]=="NN":
            bm2_dict[sku]=MLP_final(X2,y);
                                
        elif mae2["Best"][0]=="GB":
            bm2_dict[sku]=GB_final(X2,y); 
                        
        elif mae2["Best"][0]=="XG":
            bm2_dict[sku]=XG_final(X2,y);    

In [23]:
tier_pred=pd.concat([bm_dict[0][0],bm2_dict[0][0],y],axis=1)
tier_pred.columns = ['DC_Phase1', 'DC_Phase2',"Sales"]
# tier_pred.to_csv('C:\\Users\\lengada1\\NCSU\\DC_tier_predictions.csv')
tier_pred.to_csv('./DC_tier_predictions.csv')
tier_pred.round()

,DC_Phase1,DC_Phase2,Sales
0,72276.0,72564.0,72552
1,64639.0,64658.0,64297
2,63063.0,63131.0,64112
3,65082.0,64848.0,65006
4,43302.0,43272.0,43300
5,-108.0,2.0,0
6,50399.0,50397.0,50984
7,48025.0,48002.0,48246
8,47117.0,45848.0,45400
9,47812.0,47270.0,46978


In [24]:
# END NOTEBOOK
bm_dict[0][1]

XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
       colsample_bytree=1, gamma=2, learning_rate=0.01, max_delta_step=0,
       max_depth=6, min_child_weight=1, missing=None, n_estimators=900,
       n_jobs=1, nthread=None, objective='reg:linear', random_state=0,
       reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
       silent=True, subsample=1)

In [25]:
def save_model(model, filename):
    """
    Saves the model as a file. 
    - Keras models: saved as .json and .h5 files
    - scikit-learn models: saved as .pickle files
    """
    
    if type(model) is KerasRegressor:
        
        model_json = model.model.to_json()
        with open('{}.json'.format(filename), "w") as json_file:
            json_file.write(model_json)
        model.model.save_weights('{}.h5'.format(filename))        
    else:
        pickle_out = open('{}.pickle'.format(filename), "wb")
        pickle.dump(model, pickle_out)
        pickle_out.close()


# Save model to disk
filename="NCSU_DC"
model = bm_dict[0][1]
save_model(model, filename)


In [27]:
# Load model from disk
def load_pickled_model(path_to_model):
    loaded_model = pickle.load(open(path_to_model, 'rb'))
    
    return loaded_model

def load_keras_model(path_to_json, path_to_weights_h5):
    
    # load json and create model
    json_file = open(path_to_json, 'r')
    loaded_model_json = json_file.read()
    json_file.close()
    loaded_model = model_from_json(loaded_model_json)

    # load weights into new model
    loaded_model.load_weights(path_to_weights_h5)
    
    # wrap Sequential obj as a KerasRegressor to allow scikit-learn compatibility
    wrapped_loaded_model = KerasRegressor(build_fn=create_MLP, epochs=400, batch_size=20, verbose=0)
    
    return wrapped_loaded_model

def load_model(filename):
    """
    Loads a given file as a model
    - .pickle files: loaded as scikit-learn models
    - .json and .h5 files: loaded together to create Keras models
    """
    
    loaded_model = None
    
    try:
        loaded_model = load_pickled_model("{}.pickle".format(filename))
        
    except FileNotFoundError:
        loaded_model = load_keras_model("{}.json".format(filename), "{}.h5".format(filename))
        
    return loaded_model

loaded_model = load_model("NCSU_DC")

In [31]:
# Evaluate loaded model on test data
scores=[]
model_family=[]
    
# Identify the model family
if isinstance(loaded_model, RandomForestRegressor):
    model_family.append('RF')
    
elif isinstance(loaded_model, xgb.XGBRegressor):
    model_family.append('XG')   
    
elif isinstance(loaded_model, ensemble.GradientBoostingRegressor):
    model_family.append('GB')    
    
else:
    model_family.append('MLP')

# Obtain the cross validation score
score = -cross_val_score(loaded_model,X,y,cv=cv, scoring ='mean_absolute_error').mean().round(0)
scores.append(score)        

loaded_model_scores = pd.DataFrame({
    'CV Score':scores,
    'Family':model_family
})

loaded_model_scores

,CV Score,Family
0,3208.0,XG
